## This follows a CNN approach as follows

1. Convolve over each image using a filter
2. This gives us a feature map
3. At the end, use a classification head and a binary loss function

### Imports

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

from dataset_loader import TRAIN_SEGMENTATION_DATASET, TEST_SEGMENTATION_DATASET
from cnn_model import CNN_MODEL
from cnn_config import cnn_experiment_1

from trainer_evaluator import TRAINER_EVALUATOR

### Data Loader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor()
])

# Create dataset
train_dataset = TRAIN_SEGMENTATION_DATASET(
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/training/images",
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/training/ground_truth",
    transform
)

test_dataset = TEST_SEGMENTATION_DATASET(
    "/Users/tejasdhopavkar/Documents/MS/Saarland_University/Semester_1/ML/Project/dataset/test/images",
    transform
)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

# Example batch
images, labels = next(iter(train_loader))
print(images.shape, labels.shape)

torch.Size([1, 3, 375, 500]) torch.Size([1, 3, 375, 500])


### Model, criterion, optimizer

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
model = CNN_MODEL(cnn_experiment_1['model_args']['input_size'], cnn_experiment_1['model_args']['hidden_layers'], cnn_experiment_1['model_args']['activation'], cnn_experiment_1['model_args']['norm_layer'], cnn_experiment_1['model_args']['drop_prob'], cnn_experiment_1['model_args']['max_pooling']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss = nn.BCELoss()

In [5]:
model

CNN_MODEL(
  (features): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(128, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
  )
)

### Model training

In [6]:
trainer_evaluator = TRAINER_EVALUATOR()

In [7]:
print("Training started...")
trainer_evaluator.train_cnn(model, train_loader, optimizer, loss, device, 5)

Training started...
Epoch 0: Started training


RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same

### Save predictions

In [ ]:
# Save predicted masks
trainer_evaluator.save_predictions(model, train_loader, device)

TypeError: Cannot handle this data type: (1, 1, 500), |u1